In [1]:
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import numpy as np
import cv2

In [2]:
def zeroOrderConvolution(image, mask):
    imageHeight, imageWidth, numChannels = image.shape
    maskHeight, maskWidth = mask.shape
    outputHeight = imageHeight * maskHeight
    outputWidth = imageWidth * maskWidth
    output = np.zeros((outputHeight, outputWidth, numChannels), dtype=image.dtype)

    for i in range(imageHeight):
        for j in range(imageWidth):
            for k in range(maskHeight):
                for l in range(maskWidth):
                    for c in range(numChannels):
                        output[i * maskHeight + k, j * maskWidth + l, c] = image[i, j, c]
    return output

In [3]:
def firstOrderConvolution(image, mask):
    imageHeight, imageWidth, numChannels = image.shape
    maskHeight, maskWidth = mask.shape
    outputHeight = (imageHeight * 2) - 1
    outputWidth = (imageWidth * 2) - 1
    output = np.zeros((outputHeight, outputWidth, numChannels), dtype=np.float32)

    for c in range(numChannels):
        for i in range(imageHeight):
            for j in range(imageWidth):
                for k in range(maskHeight):
                    for l in range(maskWidth):
                        xOut = 2 * i + (k - 1)
                        yOut = 2 * j + (l - 1)
                        if 0 <= xOut < outputHeight and 0 <= yOut < outputWidth:
                            output[xOut, yOut, c] += image[i, j, c] * mask[k, l]
    return np.clip(output, 0, 255).astype(np.uint8)


In [4]:
def linearInterpolation(image, scale):
    origHeight, origWidth, numChannels = image.shape
    newHeight = int(origHeight * scale)
    newWidth = int(origWidth * scale)
    
    output = np.zeros((newHeight, newWidth, numChannels), dtype=np.uint8)
    rowScale = origHeight / newHeight
    colScale = origWidth / newWidth

    for c in range(numChannels):
        for y in range(newHeight):
            for x in range(newWidth):
                srcY = y * rowScale
                srcX = x * colScale
                y0 = int(np.floor(srcY))
                y1 = min(y0 + 1, origHeight - 1)
                x0 = int(np.floor(srcX))
                x1 = min(x0 + 1, origWidth - 1)

                dy = srcY - y0
                dx = srcX - x0

                topLeft = image[y0, x0, c]
                topRight = image[y0, x1, c]
                bottomLeft = image[y1, x0, c]
                bottomRight = image[y1, x1, c]

                top = (1 - dx) * topLeft + dx * topRight
                bottom = (1 - dx) * bottomLeft + dx * bottomRight
                output[y, x, c] = (1 - dy) * top + dy * bottom
    return output


In [5]:
def zeroOrderEnlarge(image):
    mask = np.array([[1, 1], [1, 1]])
    return zeroOrderConvolution(image, mask)

In [6]:
def firstOrderEnlarge(image):
    mask = np.array([[1/4, 1/2, 1/4],
                     [1/2, 1,   1/2],
                     [1/4, 1/2, 1/4]])
    return firstOrderConvolution(image, mask)

In [7]:
class ImageEnlargementApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Image Enlargement GUI")

        # Select operation type
        self.op_type = tk.StringVar()
        tk.Label(root, text="Select Enlargement Type:").pack()
        tk.Radiobutton(root, text="Zero-Order Enlargement", variable=self.op_type, value="zero").pack()
        tk.Radiobutton(root, text="First-Order Enlargement", variable=self.op_type, value="first").pack()
        tk.Radiobutton(root, text="Linear Interpolation", variable=self.op_type, value="linear").pack()

        # Choose file button
        tk.Button(root, text="Choose Image", command=self.choose_image).pack()

        # Input for k or EF
        self.k_label = tk.Label(root, text="Enter k or EF (enlargement factor):")
        self.k_label.pack()
        self.k_entry = tk.Entry(root)
        self.k_entry.pack()

        # Process button
        tk.Button(root, text="Process Image", command=self.process_image).pack()
        # Label to display image size
        self.size_label = tk.Label(root, text="Image Size: Not Available")
        self.size_label.pack()
        
    def choose_image(self):
        file_path = filedialog.askopenfilename()
        if file_path:
            self.image_path = file_path
            image = Image.open(file_path)
            self.original_image = image

    def process_image(self):
        if not hasattr(self, 'image_path'):
            messagebox.showerror("Error", "No image selected.")
            return

        k = self.k_entry.get()
        if not k.replace('.', '', 1).isdigit():
            messagebox.showerror("Error", "Invalid input for k/EF.")
            return

        k = float(k)
        image = Image.open(self.image_path)
        image_np = np.array(image)

        op_type = self.op_type.get()
        if op_type == "zero":
            enlarged_image = image_np
            for _ in range(int(k)):
                enlarged_image = zeroOrderEnlarge(enlarged_image)
        elif op_type == "first":
            enlarged_image = image_np
            for _ in range(int(k)):
                enlarged_image = firstOrderEnlarge(enlarged_image)
        elif op_type == "linear":
            enlarged_image = linearInterpolation(image_np, k)
        else:
            messagebox.showerror("Error", "Invalid operation type.")
            return

        self.show_enlarged_image(enlarged_image)

    def show_enlarged_image(self, image_np):
        # Convert numpy array to PIL Image
        result_image_pil = Image.fromarray(image_np)

        # Create a new window
        result_window = tk.Toplevel(self.root)
        result_window.title("Enlarged Image")

        # Display the image in the new window
        tk_image = ImageTk.PhotoImage(result_image_pil)
        image_label = tk.Label(result_window, image=tk_image)
        image_label.image = tk_image
        image_label.pack()
        width, height = result_image_pil.size
        self.size_label.config(text=f"Image Size: {width} x {height}")


root = tk.Tk()
app = ImageEnlargementApp(root)
root.mainloop()
